# MatchColumnsToTrainSet
<p>The MatchColumnsToTrainSet() ensure that columns in test dataset are similar to train dataset. 
If needed it drops columns that are in test but not in the train dataset and add columns that are missing 
in test dataset (but are in train).</p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from feature_engine.preprocessing import MatchColumnsToTrainSet

In [4]:
# Load titanic dataset from OpenML

def load_titanic():
    data = pd.read_csv('https://www.openml.org/data/get_csv/16826755/phpMYEkMl')
    data = data.replace('?', np.nan)
    data['cabin'] = data['cabin'].astype(str).str[0]
    data['pclass'] = data['pclass'].astype('O')
    data['age'] = data['age'].astype('float')
    data['fare'] = data['fare'].astype('float')
    data['embarked'].fillna('C', inplace=True)
    data.drop(labels=['boat', 'body', 'home.dest', 'name', 'age', 'sibsp', 'parch'], axis=1, inplace=True)
    return data

In [7]:
data = load_titanic()
data.head()

,pclass,survived,sex,ticket,fare,cabin,embarked
0,1,1,female,24160,211.3375,B,S
1,1,1,male,113781,151.5500,C,S
2,1,0,female,113781,151.5500,C,S
3,1,0,male,113781,151.5500,C,S
4,1,0,female,113781,151.5500,C,S


In [15]:
train = data.iloc[0:1000, :]
test = data.iloc[1000:, :]

columns_matcher = MatchColumnsToTrainSet(missing_values="ignore")

## 1 - If some columns are missing in test dataset

In [16]:
test_with_missing_columns = test.drop(["sex", "ticket"], axis=1)

test_with_missing_columns.head()

,pclass,survived,fare,cabin,embarked
1000,3,1,7.7500,n,Q
1001,3,1,23.2500,n,Q
1002,3,1,23.2500,n,Q
1003,3,1,23.2500,n,Q
1004,3,1,7.7875,n,Q


In [17]:
columns_matcher.fit(train)

res = columns_matcher.transform(test_with_missing_columns)

res.head()

The following variables are added to the DataFrame: ['sex', 'ticket']


,pclass,survived,sex,ticket,fare,cabin,embarked
1000,3,1,NaN,NaN,7.7500,n,Q
1001,3,1,NaN,NaN,23.2500,n,Q
1002,3,1,NaN,NaN,23.2500,n,Q
1003,3,1,NaN,NaN,23.2500,n,Q
1004,3,1,NaN,NaN,7.7875,n,Q


In this case, the columns "sex" and "ticket" that where missing in test dataset are created with missing values.

## 2 - If test contain columns not present in train

In [18]:
test_with_additional_columns = test.copy()

test_with_additional_columns.loc[:, "new_col1"] = 5
test_with_additional_columns.loc[:, "new_col2"] = "test"

test_with_additional_columns.head()

,pclass,survived,sex,ticket,fare,cabin,embarked,new_col1,new_col2
1000,3,1,male,367228,7.7500,n,Q,5,test
1001,3,1,female,367226,23.2500,n,Q,5,test
1002,3,1,female,367226,23.2500,n,Q,5,test
1003,3,1,male,367226,23.2500,n,Q,5,test
1004,3,1,female,330932,7.7875,n,Q,5,test


In [19]:
columns_matcher.fit(train)

res = columns_matcher.transform(test_with_additional_columns)

res.head()

The following variables are dropped from the DataFrame: ['new_col2', 'new_col1']


,pclass,survived,sex,ticket,fare,cabin,embarked
1000,3,1,male,367228,7.7500,n,Q
1001,3,1,female,367226,23.2500,n,Q
1002,3,1,female,367226,23.2500,n,Q
1003,3,1,male,367226,23.2500,n,Q
1004,3,1,female,330932,7.7875,n,Q


Columns that were present in test dataset but not in train dataset have been dropped.

## 3 - If test contains additionals columns but lack some others

This use case is just the concatenation of the case 1 and 2.

In [21]:
test_with_additional_and_missing_columns = test.drop(["sex", "ticket"], axis=1)

test_with_additional_and_missing_columns.loc[:, "new_col1"] = 5
test_with_additional_and_missing_columns.loc[:, "new_col2"] = "test"

test_with_additional_and_missing_columns.head()

,pclass,survived,fare,cabin,embarked,new_col1,new_col2
1000,3,1,7.7500,n,Q,5,test
1001,3,1,23.2500,n,Q,5,test
1002,3,1,23.2500,n,Q,5,test
1003,3,1,23.2500,n,Q,5,test
1004,3,1,7.7875,n,Q,5,test


In [22]:
columns_matcher.fit(train)

res = columns_matcher.transform(test_with_additional_and_missing_columns)

res.head()

The following variables are added to the DataFrame: ['sex', 'ticket']
The following variables are dropped from the DataFrame: ['new_col2', 'new_col1']


,pclass,survived,sex,ticket,fare,cabin,embarked
1000,3,1,NaN,NaN,7.7500,n,Q
1001,3,1,NaN,NaN,23.2500,n,Q
1002,3,1,NaN,NaN,23.2500,n,Q
1003,3,1,NaN,NaN,23.2500,n,Q
1004,3,1,NaN,NaN,7.7875,n,Q


Additional columns have been dropped and columns that were missing have been created with missing values.